In [1]:
import io
import pandas as pd
import requests
from datetime import datetime as dt
import matplotlib.pyplot as plt

In [2]:
file_url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_170624.txt"
s = requests.get(file_url).content
turns_df = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [3]:
len(turns_df)

197416

In [4]:
turns_df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS                                                               '],
      dtype='object')

In [5]:
turns_df.rename(columns={column:column.strip() for column in turns_df.columns}, inplace=True)
turns_df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [6]:
turns_df["DATE_TIME"] = pd.to_datetime(turns_df.DATE + " " + turns_df.TIME, format="%m/%d/%Y %H:%M:%S")
turns_stations = turns_df 
turns_stations[["REAL_ENTRIES", "REAL_EXITS"]] = (turns_stations.groupby(["C/A", "UNIT", "SCP", "STATION"])
                                                  ["ENTRIES", "EXITS"]
                                                  .transform(lambda x: x - x.shift(1)))
turns_stations.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,REAL_ENTRIES,REAL_EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,00:00:00,REGULAR,6224816,2107317,2017-06-17 00:00:00,NaN,NaN
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,04:00:00,REGULAR,6224850,2107322,2017-06-17 04:00:00,34.0,5.0
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,08:00:00,REGULAR,6224885,2107352,2017-06-17 08:00:00,35.0,30.0
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,12:00:00,REGULAR,6225005,2107452,2017-06-17 12:00:00,120.0,100.0
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,16:00:00,REGULAR,6225248,2107513,2017-06-17 16:00:00,243.0,61.0


In [7]:
# Percentage of NaN in real entries or real exits overall
100 * len(turns_stations[(turns_stations["REAL_ENTRIES"].isnull()) | (turns_stations["REAL_EXITS"].isnull())])/len(turns_stations)

2.380759411597844

In [8]:
turns_stations.dropna(subset=["REAL_ENTRIES", "REAL_EXITS"], axis=0, inplace=True)
turns_stations["TOTAL_REAL"] = turns_stations["REAL_ENTRIES"] + turns_stations["REAL_EXITS"]
turns_stations.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,REAL_ENTRIES,REAL_EXITS,TOTAL_REAL
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,04:00:00,REGULAR,6224850,2107322,2017-06-17 04:00:00,34.0,5.0,39.0
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,08:00:00,REGULAR,6224885,2107352,2017-06-17 08:00:00,35.0,30.0,65.0
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,12:00:00,REGULAR,6225005,2107452,2017-06-17 12:00:00,120.0,100.0,220.0
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,16:00:00,REGULAR,6225248,2107513,2017-06-17 16:00:00,243.0,61.0,304.0
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/17/2017,20:00:00,REGULAR,6225548,2107565,2017-06-17 20:00:00,300.0,52.0,352.0


In [9]:
# Percentage of negative entries in real entries overall
100 * len(turns_stations[turns_stations["REAL_ENTRIES"] < 0])/len(turns_stations)

0.8302372402914133

In [10]:
# Percentage of negative entries in real exits overall
100 * len(turns_stations[turns_stations["REAL_EXITS"] < 0])/len(turns_stations)

0.6600386060316735

In [11]:
# Percentage of negative entries in real entries per station
100 * (turns_stations[turns_stations["REAL_ENTRIES"] < 0].groupby(["STATION"]).size()/turns_stations.groupby(["STATION"])
 .size()).dropna().sort_values(ascending = False).head(10)

STATION
6 AV               20.098039
104 ST             18.836565
174 ST             16.666667
EASTCHSTER/DYRE    14.634146
GRAND-NEWTOWN       8.367347
57 ST-7 AV          7.253886
125 ST              6.958474
3 AV-149 ST         6.677524
BROOKLYN BRIDGE     6.029412
FRANKLIN AV         5.907781
dtype: float64

In [12]:
# Percentage of negative entries in real exits per station
100 * (turns_stations[turns_stations["REAL_EXITS"] < 0].groupby(["STATION"]).size()/turns_stations.groupby(["STATION"])
 .size()).dropna().sort_values(ascending = False).head(10)

STATION
AVENUE I           25.000000
6 AV               19.607843
163 ST-AMSTERDM    16.666667
174 ST             16.666667
HIGH ST            16.666667
57 ST-7 AV         14.507772
104 ST             13.573407
75 AV              10.024450
18 AV               9.090909
GRAND-NEWTOWN       8.367347
dtype: float64

In [13]:
#Droping all negative entries  
turns_stations = (turns_stations.drop(turns_stations
                    [(turns_stations["REAL_EXITS"] < 0) | (turns_stations["REAL_ENTRIES"] < 0)].index))

In [14]:
# Where and when we have more people
(turns_stations.groupby(["STATION","DATE_TIME"])).sum().sort_values("TOTAL_REAL", ascending = False).head(10)

ENTRIES       EXITS  REAL_ENTRIES  \
STATION         DATE_TIME                                                   
PROSPECT PARK   2017-06-19 12:00:00  1606303080   823278675  1.566070e+09   
1 AV            2017-06-18 16:00:00  3039154335  1638512980  1.621025e+09   
HARRISON        2017-06-20 12:36:36      209353     2256196  1.859180e+05   
34 ST-HERALD SQ 2017-06-20 20:00:00  1642007860  1328545792  5.281700e+04   
                2017-06-21 20:00:00  1642131325  1328657406  5.380500e+04   
                2017-06-22 20:00:00  1642254033  1328769237  5.204300e+04   
                2017-06-19 20:00:00  1641890693  1328440336  4.979000e+04   
34 ST-PENN STA  2017-06-19 20:00:00   728856628   355389908  3.525900e+04   
34 ST-HERALD SQ 2017-06-23 20:00:00  1642374569  1328881791  4.713600e+04   
34 ST-PENN STA  2017-06-21 20:00:00   729092004   355602970  3.498400e+04   

                                      REAL_EXITS    TOTAL_REAL  
STATION         DATE_TIME                                       
PROSPECT PARK   2017-06-19 12:00:00  785939439.0  2.352009e+09  
1 AV            2017-06-18 16:00:00  167108395.0  1.788133e+09  
HARRISON        2017-06-20 12:36:36    2078346.0  2.264264e+06  
34 ST-HERALD SQ 2017-06-20 20:00:00      32987.0  8.580400e+04  
                2017-06-21 20:00:00      31856.0  8.566100e+04  
                2017-06-22 20:00:00      31942.0  8.398500e+04  
                2017-06-19 20:00:00      32990.0  8.278000e+04  
34 ST-PENN STA  2017-06-19 20:00:00      45476.0  8.073500e+04  
34 ST-HERALD SQ 2017-06-23 20:00:00      32088.0  7.922400e+04  
34 ST-PENN STA  2017-06-21 20:00:00      42936.0  7.792000e+04

In [15]:
#Droping stations with a number of entries larger than 100000
turns_stations = (turns_stations.drop(turns_stations[turns_stations["REAL_EXITS"] > 100000].index))

In [16]:
turns_stations_days = (turns_stations.groupby(["STATION","DATE"])).sum().sort_values("TOTAL_REAL", ascending = False).reset_index()

In [17]:
turns_stations_days["STATION"].head(50).unique()

array(['34 ST-PENN STA', 'GRD CNTRL-42 ST', '34 ST-HERALD SQ',
       'TIMES SQ-42 ST', '23 ST', '14 ST-UNION SQ', 'FULTON ST',
       '42 ST-PORT AUTH', '86 ST', '59 ST'], dtype=object)